# WeRateDogs Twitter Analysis

This project was done as a Udacity project to put together the concepts we learned. The Twitter handle [WeRateDogs](twitter.com/dog_rates). The key thing this channel does that this project was focused on is extracting the ratings, most of which are 12/10 or some other rating over 10.

As a disclaimer, Udacity is absolute garbage and I would not recomend it to anyone as there is no support. 

## Step 1 - Gathering the Data
There are three data sources and their details are listed below:
* `img_predictions`: This is a file that was gather from Udacity using requests package. The contents include tweet_data with AI predictions as to what the dogs are in the images. 
* `post_archive`: This is a file that was provided from Udacity for download to local machine and included 5,000 posts that has been modified to pull out the dog ratings from the text strings of the posts. Udacity intentionally left key metrics out of this document.
* `tweet_data`: This data source pulls from Twitter's API directly using the tweet_id variable from post_archive. 

The code block below imports our python modules as well as our secrete API credentials so that this notebook can utilize a public git repo.

In [92]:
import requests
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import os.path
import API_Key_Credentials as creds

consumer_key = creds.consumer_key
consumer_secret = creds.consumer_secret
access_token = creds.access_token
access_secret = creds.access_secret

**Gathering** `img_predictions`

In [93]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [94]:
response.status_code

200

In [95]:
if response.status_code == 200:
    print("We're in boys")
else:
    print("Houston, we have a problem, Error: ",response.status_code)

We're in boys


In [96]:
with open('image_predictions.tsv', 'wb') as file:
    file.write(response.content)

In [97]:
img_predictions = pd.read_csv('image_predictions.tsv', delimiter='\t')
img_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1879,846514051647705089,https://pbs.twimg.com/media/C79sB4xXwAEvwKY.jpg,2,golden_retriever,0.650003,True,Leonberg,0.065199,True,Norfolk_terrier,0.052955,True
657,682303737705140231,https://pbs.twimg.com/media/CXgHoLnWAAA8i52.jpg,1,seat_belt,0.997659,False,Lakeland_terrier,0.001731,True,Airedale,0.000204,True
764,688916208532455424,https://pbs.twimg.com/media/CY-Fn1FWEAQhzhs.jpg,1,Pembroke,0.430544,True,red_fox,0.206576,False,Pomeranian,0.154352,True
901,700002074055016451,https://pbs.twimg.com/media/CbboKP4WIAAw8xq.jpg,1,Chihuahua,0.369488,True,schipperke,0.243367,True,pug,0.161614,True
134,668484198282485761,https://pbs.twimg.com/media/CUbu1GAWsAEH3E-.jpg,1,standard_poodle,0.587372,True,Bedlington_terrier,0.182411,True,Afghan_hound,0.040968,True


**Gathering** `post_archive`

In [98]:
post_archive = pd.read_csv('twitter-archive-enhanced.csv')
post_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1915,674318007229923329,NaN,NaN,2015-12-08 20:01:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Lennon. He's in quite the predicament....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674318007...,8,10,Lennon,None,None,pupper,None
832,768596291618299904,NaN,NaN,2016-08-24 23:50:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Oakley and Charlie. They're convi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/768596291...,12,10,Oakley,None,None,None,None
402,824325613288833024,NaN,NaN,2017-01-25 18:38:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Retweet the h*ck out of this 13/10 pupper #Bel...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/824325613...,13,10,None,None,None,pupper,None
1481,693280720173801472,NaN,NaN,2016-01-30 03:52:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sadie and her 2 pups Shebang &amp; Ruf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/693280720...,10,10,Sadie,None,None,None,None
882,760290219849637889,NaN,NaN,2016-08-02 01:44:48 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Rocco. He's doing his best. 13/10 some...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/760290219...,13,10,Rocco,None,None,None,None


**Gathering** `tweet_data`

In [99]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

tweet_ids = post_archive.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()


# Save each tweet's returned JSON as a new line in a .txt file
if os.path.exists('tweet_json.txt'):
    print('Twitter Info Already Captured')
else:
    with open('tweet_json.txt', 'w') as outfile:
        # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
        for tweet_id in tweet_ids:
            count += 1
            print(str(count) + ": " + str(tweet_id))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                print("Success")
                json.dump(tweet._json, outfile)
                outfile.write('\n')
            except tweepy.TweepError as e:
                print("Fail")
                fails_dict[tweet_id] = e
                pass
end = timer()


Twitter Info Already Captured


This codeblock below took a while to figure out but was my solution to storing the raw JSON from a txt file to python object. It stores each JSON as a dictionary inside the `tweet_data` list. Note that `for line in f:` is a special sytax that parses each line of the txt file.

In [100]:
with open('tweet_json.txt') as f:
    tweet_data = []
    for line in f:
        tweet_data.append(json.loads(line))
    

In [101]:
type(tweet_data)

list

In [102]:
type(tweet_data[1])

dict

Example of the raw API with all it's attributes before selectively choosing which ones to add to our  master dataFrame

In [103]:
tweet_data[4]

{'created_at': 'Sat Jul 29 16:00:24 +0000 2017',
 'id': 891327558926688256,
 'id_str': '891327558926688256',
 'full_text': 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f',
 'truncated': False,
 'display_text_range': [0, 138],
 'entities': {'hashtags': [{'text': 'BarkWeek', 'indices': [129, 138]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 891327551943041024,
    'id_str': '891327551943041024',
    'indices': [139, 162],
    'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg',
    'url': 'https://t.co/AtUZn91f7f',
    'display_url': 'pic.twitter.com/AtUZn91f7f',
    'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1',
    'type': 'photo',
    'sizes': {'medium': {'w': 720, 'h': 540, 'resize': 'fit'},
     'large': {'w': 720, 'h':

In [104]:
tweet_data[4]['retweet_count']

8837

## Step 2 - Creating Master dataFrame
This process is all about merging our three datasets then going through the process of accessing and cleaning.

This codeblock transforms our list of dictionaries to a dataFrame. Some did not contain media which would throw an error. Because in this instance we are only interested in posts with photos using a simple pass funtion solved our issue appropiately.

In [105]:
tweets = pd.DataFrame()

for i in range(1,len(tweet_data)):
    try:
        tweets = tweets.append({'tweet_id': tweet_data[i]['id'],
                                'retweets': tweet_data[i]['retweet_count'],
                                'favorite_counts': tweet_data[i]['favorite_count'],
                                'created_at': tweet_data[i]['created_at'],
                                'full_text':tweet_data[i]['full_text'].split('https://')[0],
                                'post_link':tweet_data[i]['full_text'].split('https://')[1],
                                'picture':tweet_data[i]['entities']['media'][0]['media_url'],
                                'retweeted':tweet_data[i]['retweeted']
                               }, ignore_index=True)
    except:
        pass

In [106]:
tweets.columns.tolist()

['created_at',
 'favorite_counts',
 'full_text',
 'picture',
 'post_link',
 'retweeted',
 'retweets',
 'tweet_id']

### Question:
Is it possible to merge three dataFrames at the same time in a pythonic fashion?

In [107]:
master_df = pd.merge(tweets, img_predictions, on='tweet_id')
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,Mon Jul 31 00:18:03 +0000 2017,24183.0,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,0.0,3925.0,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
1,Sun Jul 30 15:58:51 +0000 2017,40629.0,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,0.0,8161.0,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2,Sat Jul 29 16:00:24 +0000 2017,38862.0,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,0.0,8837.0,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
3,Sat Jul 29 00:08:17 +0000 2017,19532.0,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,0.0,2940.0,8.910880e+17,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
4,Fri Jul 28 00:22:40 +0000 2017,62928.0,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,0.0,17831.0,8.907292e+17,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True
5,Thu Jul 27 16:25:51 +0000 2017,26866.0,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,0.0,4041.0,8.906092e+17,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True
6,Wed Jul 26 00:31:25 +0000 2017,29605.0,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,0.0,6925.0,8.900066e+17,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True
7,Tue Jul 25 00:10:02 +0000 2017,26144.0,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,0.0,4251.0,8.896388e+17,https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,1,French_bulldog,0.991650,True,boxer,0.002129,True,Staffordshire_bullterrier,0.001498,True
8,Mon Jul 24 00:19:32 +0000 2017,24343.0,This is Oliver. You're witnessing one of his m...,http://pbs.twimg.com/ext_tw_video_thumb/889278...,t.co/WpHvrQedPb,0.0,5063.0,8.892788e+17,https://pbs.twimg.com/ext_tw_video_thumb/88927...,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True
9,Sun Jul 23 00:22:39 +0000 2017,28106.0,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,0.0,4246.0,8.889172e+17,https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,1,golden_retriever,0.714719,True,Tibetan_mastiff,0.120184,True,Labrador_retriever,0.105506,True


In [108]:
master_df = pd.merge(master_df, post_archive, on='tweet_id')
master_df.sample(20)

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
786,Sun Jan 31 03:49:30 +0000 2016,2607.0,Meet Freddery. He's a Westminster Toblerone. S...,http://pbs.twimg.com/media/CaBP7i9W0AAJrIs.jpg,t.co/6BS9XEip9a,0.0,422.0,6.936422e+17,https://pbs.twimg.com/media/CaBP7i9W0AAJrIs.jpg,1,Scottish_deerhound,0.111893,True,bluetick,0.074302,True,German_short-haired_pointer,0.067000,True,NaN,NaN,2016-01-31 03:49:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Freddery. He's a Westminster Toblerone. S...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/693642232...,9,10,Freddery,None,None,None,None
1306,Wed Nov 18 18:17:59 +0000 2015,182.0,12/10 gimme now,http://pbs.twimg.com/media/CUHREBXXAAE6A9b.jpg,t.co/QZAnwgnOMB,0.0,49.0,6.670441e+17,https://pbs.twimg.com/media/CUHREBXXAAE6A9b.jpg,1,golden_retriever,0.765266,True,Labrador_retriever,0.206694,True,seat_belt,0.010667,False,NaN,NaN,2015-11-18 18:17:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",12/10 gimme now https://t.co/QZAnwgnOMB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667044094...,12,10,None,None,None,None,None
1191,Thu Nov 26 18:16:16 +0000 2015,500.0,This is Andru. He made his very own lacrosse s...,http://pbs.twimg.com/media/CUwdYL5UsAAP0XX.jpg,t.co/1VJoY3OJ1F,0.0,159.0,6.699428e+17,https://pbs.twimg.com/media/CUwdYL5UsAAP0XX.jpg,1,vizsla,0.743216,True,redbone,0.217282,True,Rhodesian_ridgeback,0.028473,True,NaN,NaN,2015-11-26 18:16:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Andru. He made his very own lacrosse s...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669942763...,11,10,Andru,None,None,None,None
437,Sat Jul 30 17:56:51 +0000 2016,8824.0,This is Oakley. He has no idea what happened h...,http://pbs.twimg.com/media/CooZok_WEAA7oPw.jpg,t.co/vT3JM8b989,0.0,2545.0,7.594477e+17,https://pbs.twimg.com/media/CooZok_WEAA7oPw.jpg,1,kuvasz,0.223148,True,Bedlington_terrier,0.220731,True,teddy,0.181303,False,NaN,NaN,2016-07-30 17:56:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Oakley. He has no idea what happened h...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/759447681...,11,10,Oakley,None,None,None,None
231,Sat Dec 24 03:40:19 +0000 2016,6362.0,I'm happy to inform you all that Jake is in ex...,http://pbs.twimg.com/media/C0aXTLrWQAAPuJQ.jpg,t.co/LRCTJpnCnS,0.0,1299.0,8.125031e+17,https://pbs.twimg.com/media/C0aXTLqXEAADxBi.jpg,2,loupe,0.546856,False,web_site,0.345298,False,bubble,0.010528,False,NaN,NaN,2016-12-24 03:40:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I'm happy to inform you all that Jake is in ex...,NaN,NaN,NaN,https://m.facebook.com/story.php?story_fbid=18...,13,10,None,None,None,None,None
832,Sun Jan 17 19:27:24 +0000 2016,988.0,When you stumble but recover quickly cause you...,http://pbs.twimg.com/media/CY8gWE-WsAA60ZG.jpg,t.co/PMeq6IedU7,0.0,196.0,6.888048e+17,https://pbs.twimg.com/media/CY8gWFRWUAAm1XL.jpg,3,malinois,0.199512,True,German_shepherd,0.096797,True,Saluki,0.082848,True,NaN,NaN,2016-01-17 19:27:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you stumble but recover quickly cause you...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/688804835...,12,10,None,None,None,None,None
1049,Tue Dec 08 01:47:22 +0000 2015,722.0,Yea I can't handle the cuteness anymore. Curls...,http://pbs.twimg.com/media/CVquIDRW4AEJrPk.jpg,t.co/sAI6gCGZYX,0.0,130.0,6.740426e+17,https://pbs.twimg.com/media/CVquIDRW4AEJrPk.jpg,1,toy_poodle,0.927975,True,miniature_poodle,0.068946,True,standard_poodle,0.001316,True,NaN,NaN,2015-12-08 01:47:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Yea I can't handle the cutenes

## Step 3 - Cleaning the Master dataFrame

In [109]:
master_df.columns.tolist()

['created_at',
 'favorite_counts',
 'full_text',
 'picture',
 'post_link',
 'retweeted',
 'retweets',
 'tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [110]:
#Transforms the appropiate fields to datetime format
time_variables = ['created_at','timestamp','retweeted_status_timestamp']
for var in time_variables:
    master_df[var] = pd.to_datetime(master_df[var])

In [111]:
#Checking dTypes
master_df.dtypes

created_at                    datetime64[ns, UTC]
favorite_counts                           float64
full_text                                  object
picture                                    object
post_link                                  object
retweeted                                 float64
retweets                                  float64
tweet_id                                  float64
jpg_url                                    object
img_num                                     int64
p1                                         object
p1_conf                                   float64
p1_dog                                       bool
p2                                         object
p2_conf                                   float64
p2_dog                                       bool
p3                                         object
p3_conf                                   float64
p3_dog                                       bool
in_reply_to_status_id                     float64


In [112]:
#Searching for null values
master_df.isnull().sum()

created_at                       0
favorite_counts                  0
full_text                        0
picture                          0
post_link                        0
retweeted                        0
retweets                         0
tweet_id                         0
jpg_url                          0
img_num                          0
p1                               0
p1_conf                          0
p1_dog                           0
p2                               0
p2_conf                          0
p2_dog                           0
p3                               0
p3_conf                          0
p3_dog                           0
in_reply_to_status_id         1327
in_reply_to_user_id           1327
timestamp                        0
source                           0
text                             0
retweeted_status_id           1297
retweeted_status_user_id      1297
retweeted_status_timestamp    1297
expanded_urls                    0
rating_numerator    

In [113]:
pd.set_option('display.max_columns', 500)
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,2017-07-31 00:18:03+00:00,24183.0,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,0.0,3925.0,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,NaN,NaN,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
1,2017-07-30 15:58:51+00:00,40629.0,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,0.0,8161.0,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,NaN,NaN,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
2,2017-07-29 16:00:24+00:00,38862.0,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,0.0,8837.0,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,NaN,NaN,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
3,2017-07-29 00:08:17+00:00,19532.0,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,0.0,2940.0,8.910880e+17,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,NaN,NaN,2017-07-29 00:08:17+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
4,2017-07-28 00:22:40+00:00,62928.0,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,0.0,17831.0,8.907292e+17,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,NaN,NaN,2017-07-28 00:22:40+00:00,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
5,2017-07-27 16:25:51+00:00,26866.0,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,0.0,4041.0,8.906092e+17,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True,NaN,NaN,2017-07-27 16:25:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
6,2017-07-26 00:31:25+00:00,29605.0,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,0.0,6925.0,8.900066e+17,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True,NaN,NaN,2017-07-26 00:31:25+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is K

In [114]:
master_df.drop(columns={'timestamp','jpg_url','source','text','expanded_urls','in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'}, inplace=True)

In [115]:
master_df.retweets = master_df.retweets.astype(int)
master_df.favorite_counts = master_df.favorite_counts.astype(int)
master_df.retweeted = master_df.retweeted.astype(int)
master_df.retweeted = master_df.retweeted.astype(bool)
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,2017-07-31 00:18:03+00:00,24183,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,False,3925,8.918152e+17,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,12,10,Archie,None,None,None,None
1,2017-07-30 15:58:51+00:00,40629,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,False,8161,8.916896e+17,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,13,10,Darla,None,None,None,None
2,2017-07-29 16:00:24+00:00,38862,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,False,8837,8.913276e+17,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,12,10,Franklin,None,None,None,None
3,2017-07-29 00:08:17+00:00,19532,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,False,2940,8.910880e+17,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False,13,10,None,None,None,None,None
4,2017-07-28 00:22:40+00:00,62928,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,False,17831,8.907292e+17,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True,13,10,None,None,None,None,None
5,2017-07-27 16:25:51+00:00,26866,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,False,4041,8.906092e+17,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True,13,10,Zoey,None,None,None,None
6,2017-07-26 00:31:25+00:00,29605,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,False,6925,8.900066e+17,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True,13,10,Koda,None,None,None,None
7,2017-07-25 00:10:02+00:00,26144,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,False,4251,8.896388e+17,1,French_bulldog,0.991650,True,boxer,0.002129,True,Staffordshire_bullterrier,0.001498,True,12,10,Ted,None,None,None,None
8,2017-07-24 00:19:32+00:00,24343,This is Oliver. You're witnessing one of his m...,http://pbs.twimg.com/ext_tw_video_thumb/889278...,t.co/WpHvrQedPb,False,5063,8.892788e+17,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True,13,10,Oliver,None,None,None,None
9,2017-07-23 00:22:39+00:00,28106,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,False,4246,8.889172e+17,1,golden_retriever,0.714719,True,Tibetan_mastiff,0.120184,True,Labrador_retriever,0.105506,True,12,10,Jim,None,None,None,None
